# Other

In [21]:
# human (sox9) chr17:72121019-72126416
# SOX9	SOX9	BestRefSeq	protein_coding	chr17	72121019	72126416	+	SRY-box transcription factor 9	

import pysam

fasta = pysam.FastaFile("/home/chenzonggui/labweb/data/genomes/homo_sapiens/genome.fasta")
seq = fasta.fetch("chr17", 72121019, 72126416)
fasta.close()

In [23]:
with open("human.sox9.fasta", "w+") as fw:
    fw.write(">human-sox9\n")
    fw.write("%s\n" % seq)

In [28]:
with open("map_human_sox9.chain") as f, open("map_human_sox9.bed", "w+") as fw:
    lines = None
    for line in f:
        line = line.strip("\n")
        if line.startswith("#"):
            continue
        if line.startswith("chain"):
            lines = [line]
        elif line == "":
            assert len(lines) > 1
            chain, score, tname, tsize, tstrand, tstart, tend, qname, qsize, qstrand, qstart, qend, i = lines[0].split(" ")
            print(len(lines))
            tstart = int(tstart)
            tend = int(tend)
            
            chrom = tname
            start = tstart
            end = tend
            name = "homoloy"
            strand = qstrand
            score = int(score)
            color = "0,0,255"
            block_sizes = []
            block_offsets = []
            offset = 0
            for l in lines[1:]:
                vs = l.split("\t")
                a = int(vs[0])
                block_sizes.append(a)
                block_offsets.append(offset)
                
                offset += a
                if len(vs) > 1:
                    b = int(vs[1])
                    offset += b
                # print(len(vs))
            s1 = ",".join(map(str, block_sizes)) + ","
            s2 = ",".join(map(str, block_offsets)) + ","
            s3 = "\t".join(map(str, [chrom, start, end, name, score, strand, start, start, color, len(block_sizes), s1, s2]))
            # print(s3)
            fw.write(s3 + "\n")
            
                
        else:
            lines.append(line)
        

33
22
34
32
30
36
5
4
4
3
3
6
3
4
5
5
3
3
2
5
4
2
2
2
3
6
6
2
7
2


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rcParams
import seaborn
rcParams["font.family"] = "arial"

In [98]:
dat = pd.read_csv("genome.sizes", sep="\t", header=None)
dat = dat.sort_values(by=1, ascending=False)

xs = [78093715, 37502051]
colors = ["orange", 'orange']
yticks = ["chr4 (Zebrafish)", "chr25 (Zebrafish)"]

for name, length in dat.head(10).values:
    # print(name, length)
    xs.append(length)
    yticks.append(name)
    colors.append("dodgerblue")
    
ys = np.arange(len(xs))[::-1]

xs = np.array(xs) / 1e6

plt.figure(figsize=(4, 3))
plt.barh(ys, xs, height=0.8, lw=1, edgecolor="black", color=colors)
plt.xlim(0, 100)
plt.xlabel("Length of refSeq (Mb)")
plt.ylim(-0.5, 11.5)
plt.yticks(ys, yticks)
plt.gca().spines["top"].set_visible(False)
plt.gca().spines["right"].set_visible(False)

plt.tight_layout()
# plt.show()
plt.savefig("refSeq.length.png", dpi=300)
plt.close()

In [8]:
from collections import Counter

In [4]:
dat = pd.read_csv("GCF_002260705.1_Sdu_1.0_genomic.tsv", sep="\t")
tids1 = set(dat[dat["BioType"] == "protein_coding"]["TranscriptID"].values)
tids2 = set(dat[dat["BioType"] == "lncRNA"]["TranscriptID"].values)

In [12]:
from pyBioInfo.IO.File import BedFile

In [40]:
path1 = "cpat.protein_coding.cds.bed"
path2 = "cpat.lncRNA.bed"

fw1 = open(path1, "w+")
fw2 = open(path2, "w+")
recorder1 = set()
recorder2 = set()
a, b, c, d = 0, 0, 0, 0
with BedFile("GCF_002260705.1_Sdu_1.0_genomic.sorted.bed") as bed:
    for obj in bed:
        key = (obj.chrom, obj.start, obj.end, obj.strand)
        if obj.name in tids1:
            a += 1
            if obj.thick:
                if key in recorder1:
                    continue
                b += 1
                x = obj.clip(obj.thick[0], obj.thick[1])
                fw1.write(x.format("BED") + "\n")
                recorder1.add(key)
        if obj.name in tids2:
            c += 1
            if key in recorder2:
                continue
            d += 1
            fw2.write(obj.format("BED") + "\n")
            recorder2.add(key)
fw1.close()
fw2.close()
print(a, b, c, d)

33024 26818 745 710
